<p align="center"><img width="50%" src="https://www.modelshare.ai/img/ModelShareLogo_COLOR.abf79305.png" /></p>


---




<p align="center"><h1 align="center">Quick Start: Sports Clips Video Classification using Keras</h1> 

## **Create Model Playground ML Project Space: Submit, Track, and Deploy ML Model(s) in about 5 mins**
> ### (1) Set Up Environment 
> ### (2) Create a Model Playground
> ### (3) Submit, Track, and Deploy ML Model(s)
> ### (4) Improve Models and Analyze Model Performance
> ### (5) Maintain Your Model Playground

## **(1) Set Up Environment**

In order to deploy a Model Share AI Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your Model Share AI username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://docs.aws.amazon.com/general/latest/gr/aws-sec-cred-types.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github. Model Share AI does not store your credentials.*


In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [ ]:
# Generate credentials file
from aimodelshare.aws import configure_credentials 
configure_credentials()

In [ ]:
# Set credentials 
from aimodelshare.aws import set_credentials
set_credentials(credential_file="credentials.txt", type="deploy_model")

Modelshare.ai login credentials set successfully.
AWS credentials set successfully.


In [ ]:
# # Download sports video data and and pretrained Keras models
from aimodelshare.data_sharing.download_data import import_quickstart_data
keras_model, y_train_labels = import_quickstart_data("sports")
keras_model_2, y_test, y_test_labels = import_quickstart_data("sports", "competition")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.

Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


## **(2) Create a Model Playground**

### **Create a Model Playground**


> ### Each Model Playground is a unique ML project space with lots of useful functionality...
- Use playground.submit_model() to track ML models in experiments and/or competition leaderboards
- Save, version, and reuse ML models each time they are submitted
- Use playground.compare_models() to compare model architectures with git like colorized model diffs
- Use playground.deploy_model() to deploy models instantly into live REST APIs and/or web-apps.  

*  *First, you need to intantiate a ModelPlayground() object locally. Please specify the data type of your learning task (e.g., image, tabular, etc), whether the model is a classification or a regression model, and whether your playground should be public or private. If you choose "private" only approved team members can interact your model playground*  
*  *Second, you can to create a playground page on the modelshare.ai website. You need to submit evaluation data that submitted models can be evaluated against.*  



In [ ]:
# Instantiate Model Playground object
from aimodelshare.playground import ModelPlayground
myplayground=ModelPlayground(input_type="video", 
                             task_type="classification", 
                             private=False)

# Create Model Playground Page on modelshare.ai website
myplayground.create(eval_data=y_test_labels)

Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 58 seconds. 
 Playground Url: "https://lvdkig42r8.execute-api.us-east-2.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
You can make predictions with the Dashboard and access example code from the Programmatic tab.
https://www.modelshare.ai/detail/model:3502

Check out your Model Playground page for more.


##  **3) Submit, Track, and Deploy Model(s)**


### **Write a Preprocessor Function**


> ###   Preprocessing functions play a crucial role in preparing data for your model by converting it into the specific format that is needed to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  
*  *For image prediction models users should minimally include function inputs for a video filepath and values to reshape the image height and width.*  



In [ ]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(video_file_path, num_frames=60, gap_frames=3, **kwargs):

    """
      This function preprocesses the data to extract frames out of each video and resize
      them to a fixed size of (128x128) pixels. Moreover, these images are flattened out to
      act as features for each time step.
      
      params:
          video_file_path
              location of video file to be processed

          num_frames
              the number of frames to be extracted from each video. If there aren't
              sufficient frames, then the list is padded with zeros
              
          gap_frames:
              the number of frames after which we extract the next frame. If =1,
              contiguous frames are extracted
              
      returns:
          X
              transformed features corresponding to data passed as input to model
      
    """

    import cv2
    import numpy as np

    vidcap = cv2.VideoCapture(video_file_path)
    
    frames = []

    success, frame = vidcap.read()
    idx = 0

    while success:
        # for each frame, do if we satisfy the gap_frames parameter
        if idx % gap_frames == 0:
            # convert to RGB (default cv2 is BGR)
            # this is important because the vgg model is trained on RGB images
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (128, 128))

            # extract features for the resized image
            frame = frame / 255.0 # Min-max transform.

            # flatten the features and append to a list of features for this video
            frame = frame.reshape(-1)

            frames.append(frame)

            if len(frames) >= num_frames:
                break

        idx += 1
        success, frame = vidcap.read()

    # if number of timesteps or frames < num_frames, pad with zeroes for consistent shape
    while len(frames) < num_frames:
        frames.append(np.zeros(*frames[-1].shape))

    X = np.array(frames)
    X = np.expand_dims(X, axis=0)
    X = np.array(X, dtype=np.float32)

    return X

### **Train Your Machine Learning Model**


> ### Model Share supports Scikit-Learn, Tensorflow,  Keras, and Pytorch models out of the box.  

*  *Train your model using your favorite ML library. Here, we have already loaded a Keras model in the quickstart function above.*  
*  *Use your model to make predictions on the test set.*  




In [ ]:
# Check the model architecture.
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                9840600   
                                                                 
 dense (Dense)               (None, 20)                1020      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 9,841,683
Trainable params: 9,841,683
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Preprocess X_test video data to generate predictions from models 
# Unzip video clips
import zipfile
with zipfile.ZipFile('sports_clips_competition_data/clips_test.zip', 'r') as zip_ref:
    zip_ref.extractall('X_test_clips')

# Preprocess clips
import numpy as np
file_names = [('/content/X_test_clips/clips_test/' + str(i) + '.avi') for i in range(1, 46)]
preprocessed_video_data = [preprocessor(x) for x in file_names]

#Create single X_test array from preprocessed videos
X_test = np.vstack(preprocessed_video_data) 

In [ ]:
# Generate predicted y values
prediction_column_index=keras_model.predict(X_test).argmax(axis=1)

# Extract correct prediction labels
prediction_labels = [y_train_labels.columns[i] for i in prediction_column_index]

2/2 [==============================] - 2s 615ms/step


### **Submit a Model to your Model Playground**


> ### You can submit models to your Model Playground page in order to evaluate them, extract model metadata, compare them to other models, and turn them into REST API endpoints.  

*  *Pass your model to the submit model function. You can either pass a model object, an ONNX object, or a filepath to an ONNX file.*  
*  *Pass your preprocessor to ensure that your model can process new data.*
*  *Pass your predictions to be evaluated against your playground evaluation data.*  




In [ ]:
# Submit Model to Experiment Leaderboard
# (Option: set submission_type="competition" to submit to competition leaderboard)

myplayground.submit_model(model=keras_model,
                          preprocessor=preprocessor,
                          prediction_submission=prediction_labels)


Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 
Your model has been submitted to experiment as model version 4.
Visit your Model Playground Page for more.


### **Deploy a Model to a Prediction REST API and Web-app**


> ### Update the prediction API behind your Model Playground with a new model and verify the model performance metrics in your Model Playground.

Use playground.deploy_model() with the following key arguments:
* *model_version: Simply refer to the model by its leaderboard version number.*
* *example_data: deploy_model builds a web-app for you.  It needs some example data for end users to download to try your web-app.  For image data provide the path to a folder with some example images.* 
*  *y_train: For classification problems deploy_model needs well ordered y_training data labels.  Provide a python list of y_train labels or a one hot encoded Pandas y_training data frame, so your web-app and prediction api return the correct prediction labels*

In [ ]:
# Deploy model by version number
myplayground.deploy_model(model_version=1, 
                          example_data="sports_quick_start_materials/example_data", 
                          y_train=y_train_labels)

Runtime model & preprocessor for api: https://lvdkig42r8.execute-api.us-east-2.amazonaws.com/prod/m updated to model version 1.

Model metrics are now updated and verified for this model playground.


## **(4) Improve Models and Analyze Model Performance**

### **Set Submission Credentials**

> ### Authorized users can submit new models after setting credentials using modelshare.ai username/password.  

*  *This only needs to be done if you want to submit models in a new session.*  
*  *skip this step if you have already set your deployment credentials.*  




In [ ]:
# example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m
apiurl=myplayground.playground_url 

set_credentials(apiurl=apiurl)

Modelshare.ai Username:··········
Modelshare.ai Password:··········
Modelshare.ai login credentials set successfully.


### **Submit More Models to your Model Playground**


> ### You can submit models to your Model Playground page in order to evaluate them, extract model metadata, compare them to other models, and turn them into REST API endpoints.  

*  *Pass your model to the submit model function. You can either pass a model object, an ONNX object, or a filepath to an ONNX file.*  
*  *Pass your preprocessor to ensure that your model can process new data.*
*  *Pass your predictions to be evaluated against your playground evaluation data.*  


In [ ]:
# Have a look at architecture for model 2
keras_model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 5)             983160    
                                                                 
 lstm_1 (LSTM)               (None, 5)                 220       
                                                                 
 dense (Dense)               (None, 3)                 18        
                                                                 
Total params: 983,398
Trainable params: 983,398
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Submit Model 2

# Generate predicted y values (Model 2)
prediction_column_index=keras_model_2.predict(X_test).argmax(axis=1)

# extract correct prediction labels (Model 2)
prediction_labels = [y_train_labels.columns[i] for i in prediction_column_index]

# Submit Model 2 to Experiment Leaderboard
myplayground.submit_model(model = keras_model_2,
                          preprocessor=preprocessor,
                          prediction_submission=prediction_labels)

2/2 [==============================] - 1s 199ms/step

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 
Your model has been submitted to experiment as model version 3.
Visit your Model Playground Page for more.


### **Check Experiment Leaderboard**


> ### Leaderboards allow you to compare model architectures ranked by predictive performance.  

*  *Query leaderboard data for models submitted to your Model Playground page.* 
*  *You can either stylize the leaderboard or use pandas to run your own analyses to explore which model architectures perform well.*  




In [ ]:
# Check experiment leaderboard
data = myplayground.get_leaderboard()
myplayground.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,model_type,depth,num_params,dense_layers,lstm_layers,relu_act,softmax_act,tanh_act,loss,optimizer,memory_size,username,version
0,69.57%,55.46%,52.94%,61.90%,keras,Sequential,3,983398,1,2,nan,1,2,function,Adam,3934712,mikedparrott,2
1,69.57%,55.46%,52.94%,61.90%,keras,Sequential,3,983398,1,2,nan,1,2,function,Adam,3934712,newusertest,3
2,52.17%,48.16%,45.10%,61.90%,keras,Sequential,3,9841683,2,1,1.000000,1,1,function,Adam,39367720,mikedparrott,1


### **Compare Two or More Models**


> ### Model comparisons can help you to identify differences in model architecture at a glance.  

*  *Simply pass the version numbers of the models you would like to compare.* 
*  *For Keras and Pytoch models you can compare architectures through color-coded layer types. For Scikit-Learn models, you can compare hyperparameter settings.*  

In [ ]:
# Compare two or more models
data=myplayground.compare_models([1,2], verbose=1)
myplayground.stylize_compare(data)

### **Check the Structure of Evaluation Data**

> ### Your team needs to understand the format of the evaluation data, to ensure that their submissions can be processed.  

*  *Make sure class labels, data types, and dimensions of predictions correspond to submitted evaluation data.*
*  *This is important to generate evaluation metrics for your submission.*


In [ ]:
# Check structure of evaluation data
myplayground.inspect_eval_data()

{'ytest_example': ['kayaking', 'horserace', 'pullups', 'kayaking', 'kayaking'],
 'y_length': 45,
 'class_labels': ['kayaking', 'horserace', 'pullups'],
 'class_balance': {'kayaking': 13, 'horserace': 19, 'pullups': 13},
 'label_dtypes': {"<class 'str'>": 45}}

## **(5) Maintaining Your Model Playground**

### **Update the Runtime Model (the model that is used for predictions in your REST API/Web-App**


> ### You can update the runtime model anytime by picking one of the previously submitted models.  

* *Simply refer to the model by its leaderboard version number.*
* *You can choos a model from the Competition or Experiment leaderboard by specifying the submission_type argument.*

In [ ]:
# Update runtime model
myplayground.update_runtime_model(model_version=2)

Runtime model & preprocessor for api: https://lvdkig42r8.execute-api.us-east-2.amazonaws.com/prod/m updated to model version 2.

Model metrics are now updated and verified for this model playground.


### **Delete Deployment**


> ### You can delete the entire Model Playground, including the REST API, web dashboard, competition, and all submitted models.  

* *This is final - there is no way to recover your Model Playground if you delete it.*
* *We have commented the line for safety reasons.*

In [ ]:
#myplayground.delete_deployment()